In [21]:
import networkx as nx
import pandas as pd
import plotly.graph_objects as go

datasets = ["A$APRocky.csv", "AvrilLavigne.csv", "ChiefKeef.csv", "Drake.csv", "EdSheeran.csv",
            "EllieGoulding.csv", "GucciMane.csv", "KanyeWest.csv", "KatyPerry.csv", "KidCudi.csv",
            "LanaDelRey.csv", "LUCKI.csv", "MetroBoomin.csv", "MikeDean.csv", "PlayboiCarti.csv",
            "TaylorSwift.csv", "Tyler,TheCreator.csv", "YungLean.csv"]
folderPath = 'FinalDatasets/'

def cleanText(text):
    retText = ''
    text = str(text)
    try:
        for ch in text.lower():
            if ch.isalpha() or ch == ' ' or ch.isnumeric():
                retText += ch
    except Exception as e:
        print(f'UwU yowr ewwow is: {e} fow text {text}')
    return retText

def csvToDF(location):
    df = pd.read_csv(location)
    df['Lyrics'] = cleanText(df['Lyrics'])
    return df

dataFrameArray = []

for artist in datasets:
    dataFrameArray.append(csvToDF(f'{folderPath}{artist}'))

def graphMaker(array):
    G = nx.Graph()

    unique_artists = set()
    unique_producers = set()

    for dataframe in array:
        albumDict = {}

        for _, row in dataframe.iterrows():
            artist = row['Artist']
            producer = row['Producer']

            if producer not in unique_producers:
                G.add_node(producer, node_type='producer', color='blue')
                unique_producers.add(producer)

            if artist not in unique_artists:
                G.add_node(artist, node_type='artist', color='black')
                unique_artists.add(artist)

            if G.has_edge(producer, artist):
                continue

            G.add_edge(producer, artist)
    
    print(len(unique_artists))
    print(len(unique_producers))
    pos = nx.spring_layout(G)

    # Create nodes trace
    node_trace = go.Scatter(
        x=[pos[node][0] for node in G.nodes()],
        y=[pos[node][1] for node in G.nodes()],
        mode='markers',
        text=[node for node in G.nodes()],
        hoverinfo='text',
        marker=dict(
            size=10,
            color=[G.nodes[node]['color'] for node in G.nodes()],
        )
    )

    # Create edges trace
    edge_x = []
    edge_y = []

    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        mode='lines',
        line=dict(width=1, color='black'),
        hoverinfo='none'
    )

    # Create a list of node information for hover text
    node_info = [f"{node}" for node in G.nodes()]

    # Update figure creation with correct data
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=0),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    ))

    # Add hover text
    print(G.number_of_nodes())
    fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))
    fig.show()

graphMaker(dataFrameArray)


18
2568
2569
